In [24]:
from langchain.memory import ConversationSummaryMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.1)

memory = ConversationSummaryMemory(llm=llm)


def add_message(input, output):
    memory.save_context({"input": input}, {"output": output})


def get_history():
    return memory.load_memory_variables({})


add_message(
    "Hi I'm Shin Woo Lee, I live in South Korea", "Wow that is so cool!"
)

In [25]:
add_message("South Korea is so pretty", "I wish I could go!!")

In [26]:
get_history()

{'history': 'The human introduces themselves as Shin Woo Lee from South Korea. The AI responds by expressing admiration for this information and wishes it could visit South Korea because it is so pretty.'}